In [0]:
!git clone https://github.com/hadifar/GrammarCorrection.git

In [0]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get update -qq 2>&1 > /dev/null
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [7]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
# Work around misordering of STREAM and STDIN in Jupyter.
# https://github.com/jupyter/notebook/issues/3159
prompt = !google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass(prompt[0] + '\n\nEnter verification code: ')
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force

Enter verification code: ··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [10]:
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Files in Drive:')
!ls drive/
!ls drive/grammar/

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option
Files in Drive:
drive/grammar/final-train.tok.trg
drive/grammar/final-train.tok.trg


In [0]:
!ls GrammarCorrection

!python GrammarCorrection/train/prepare_data.py --text_A="drive/grammar/final-train.tok.trg" --text_B="drive/grammar/final-train.tok.src" --out_file="drive/grammar/trg_src_prepped.h5"


In [30]:
!ls drive/grammar/
!mkdir drive/grammar/weights/
!ls drive/grammar

final-train.tok.src  final-train.tok.trg  trg_src_prepped.h5  weights
mkdir: cannot create directory ‘drive/grammar/weights/’: File exists
final-train.tok.src  final-train.tok.trg  trg_src_prepped.h5  weights


In [0]:
!python GrammarCorrection/train/train.py --dataset="drive/grammar/trg_src_prepped.h5" --weights_path="drive/grammar/weigths/KerasAttentionNMT.h5"

In [0]:
!python GrammarCorrection/train/predict.py --dataset="drive/grammar/trg_src_prepped.h5" --weights_path="drive/grammar/weigths/KerasAttentionNMT.h5"